# SEIRHVD model example
## Work in progress (equations not ready)
\begin{align}
\dot{S} & = S_f - \alpha\beta\frac{SI}{N+k_I I+k_R R} + r_{R\_S} R\\
\dot{E} & = E_f + \alpha\beta\frac{SI}{N+k_I I+k_R R} - E\frac{1}{t_{E\_I}} \\
\dot{I} & = I_f + E\frac{1}{t_{E\_I}} - I\frac{1}{t_{I\_R}} \\
\dot{R} & = R_f + I\frac{1}{t_{I\_R}} - r_{I\_R} R\\
\end{align}
Where:  
* $S:$ Susceptible
* $E:$ Exposed
* $I:$ Infectious
* $R:$ Removed
* $\alpha:$ Mobilty
* $\beta:$ Infection rate
* $N:$ Total population
* $t_{E\_I}:$ # Transition time between exposed and infectious
* $t_{I\_R}:$ # Transition time between infectious and recovered
* $r_{R\_S}:$ Immunity loss rate ($\frac{1}{t_{R\_S}}$)  
* $S_f,E_f,I_f,R_f:$ External flux
* $k_I:$ Infected saturation   
* $k_R:$ Immunity shield    


In [1]:
# Util libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Adding lib paths

# cv19 libraries
from cv19gm.models.seirhvd import SEIRHVD  
from cv19gm.utils import cv19functions

In [2]:
# For pop-up plots execute this code (optional)
import platform
OS = platform.system()

if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
elif OS == 'Darwin':
    %matplotlib tk
    print('Mac (Funciona?)')

Linux


# Variable CFR 

## Discrete change
* pH_r de 0.7 a 0.6
* pH_D de 0.3 a 0.4

In [3]:
pH_R = cv19functions.events(values=[0.7,0.5],days=[[0,30],[30,500]])
pH_D = cv19functions.events(values=[0.3,0.5],days=[[0,30],[30,500]])

In [5]:
%%capture
# Input configuration file
config = 'cfg/SEIRHVD.toml'
# Build simulation object
model1 = SEIRHVD(config = config, H_cap=4000, pH_R = pH_R,pH_D = pH_D)
# Simulate (solve ODE)
model1.solve()

In [6]:
t = np.linspace(0,50,1000)
plt.plot(t,100*pH_R(t),label='pH_R')
plt.plot(t,100*pH_D(t),label='pH_D')
plt.plot(t,100*pH_D(t)*model1.pE_Icr(t),label='CFR')
plt.legend(loc=0)
plt.title('CFR change (%)')
plt.show()

In [7]:
t = model1.t
plt.plot(t,100*model1.pH_R(t),label='pH_R')
plt.plot(t,100*model1.pH_D(t),label='pH_D')
plt.plot(t,100*model1.CFR,label='CFR')
plt.xlim(0,50)
plt.legend(loc=0)
plt.title('CFR change (%)')
plt.show()

* Nota: la transicion pareciera ocurrir en 2 días, pero tiee que ver con la resolución en la cual se entregan los datos, la transición que utiliza el integrador es "instantánea".

In [8]:
# Plot matplotlib
fig, axs = plt.subplots(figsize=(13,9),linewidth=5,edgecolor='black',facecolor="white")
axs2 = axs.twinx()

axs.plot(model1.t,model1.D_d,color='tab:red',label='Daily deaths')
axs.set_ylabel('Deaths',color='tab:red')
axs.tick_params(axis='y', labelcolor='tab:red')

t = model1.t
axs2.plot(t,100*pH_D(t)*model1.pE_Icr(t),color='tab:blue',label='CFR')
axs2.set_ylabel('CFR',color='tab:blue')
axs2.tick_params(axis='y',labelcolor='tab:blue')

axs.set_xlim(0,200)
axs2.set_xlim(0,200)
fig.legend(loc=8)
fig.suptitle('CFR vs Deaths')
fig.show()

## Continious change
* pH_r de 0.7 a 0.6
* pH_D de 0.3 a 0.4

In [9]:
pH_R = cv19functions.sigmoidal_transition(t_init=20,t_end=40,initvalue = 0.7, endvalue = 0.5)
pH_D = cv19functions.sigmoidal_transition(t_init=20,t_end=40,initvalue = 0.3, endvalue = 0.5)

In [11]:
%%capture
# Input configuration file
config = 'cfg/SEIRHVD.toml'
# Build simulation object
model2 = SEIRHVD(config = config, H_cap=4000, pH_R = pH_R,pH_D = pH_D)
# Simulate (solve ODE)
model2.solve()

In [12]:
t = model2.t
plt.plot(t,100*model2.pH_R(t),label='pH_R')
plt.plot(t,100*model2.pH_D(t),label='pH_D')
plt.plot(t,100*model2.CFR,label='CFR')
plt.xlim(0,50)
plt.legend(loc=0)
plt.title('CFR change (%)')
plt.show()

In [13]:
# Plot matplotlib
fig, axs = plt.subplots(figsize=(13,9),linewidth=5,edgecolor='black',facecolor="white")
axs2 = axs.twinx()

axs.plot(model2.t,model2.D_d,color='tab:red',label='Daily deaths')
axs.set_ylabel('Deaths',color='tab:red')
axs.tick_params(axis='y', labelcolor='tab:red')

t = model2.t
axs2.plot(t,100*pH_D(t)*model2.pE_Icr(t),color='tab:blue',label='CFR')
axs2.set_ylabel('CFR',color='tab:blue')
axs2.tick_params(axis='y',labelcolor='tab:blue')

axs.set_xlim(0,200)
axs2.set_xlim(0,200)
fig.legend(loc=8)
fig.suptitle('CFR vs Deaths')
fig.show()

# Access CFR value

## As a variable

In [14]:
model2.CFR 

array([0.01500006, 0.01500009, 0.01500014, 0.0150002 , 0.0150003 ,
       0.01500045, 0.01500068, 0.01500101, 0.01500151, 0.01500225,
       0.01500335, 0.015005  , 0.01500746, 0.01501113, 0.01501659,
       0.01502473, 0.01503684, 0.01505486, 0.01508163, 0.01512128,
       0.01517986, 0.01526597, 0.01539166, 0.01557324, 0.01583173,
       0.01619203, 0.01667982, 0.01731475, 0.01810026, 0.01901312,
       0.02      , 0.02098688, 0.02189974, 0.02268525, 0.02332018,
       0.02380797, 0.02416827, 0.02442676, 0.02460834, 0.02473403,
       0.02482014, 0.02487872, 0.02491837, 0.02494514, 0.02496316,
       0.02497527, 0.02498341, 0.02498887, 0.02499254, 0.024995  ,
       0.02499665, 0.02499775, 0.02499849, 0.02499899, 0.02499932,
       0.02499955, 0.0249997 , 0.0249998 , 0.02499986, 0.02499991,
       0.02499994, 0.02499996, 0.02499997, 0.02499998, 0.02499999,
       0.02499999, 0.02499999, 0.025     , 0.025     , 0.025     ,
       0.025     , 0.025     , 0.025     , 0.025     , 0.025  

## As part of the pandas array

In [15]:
model2.results['CFR']

0      0.015
1      0.015
2      0.015
3      0.015
4      0.015
       ...  
495    0.025
496    0.025
497    0.025
498    0.025
499    0.025
Name: CFR, Length: 500, dtype: float64